# Data Cleaning
In this stage, you will read in the raw data and perform any necessary reshaping, combining, and cleaning of the data. Though this stage is not explicitly an exploratory stage, you will learn a lot about the content of the data throughout the course of this notebook.

### Imports

For your convenience, we have included a few pre-written functions, which you might find useful in your analysis. They are by no means necessary, but feel free to use any or all of them. The code for these functions can be found in /src

In [1]:
import pandas as pd
import os
import sys

In [2]:
src_path = os.path.abspath('../src/')
sys.path.append(src_path)

from data_cleaning import *

The Data Cleaning functions in the src file are:

- standardize_col_names(df)

Standardizes column names of a dataframe. It will remove white space, replace spaces with underscores, and eliminate special characters (including parenthesis and slashes). All letters are converted to lowercase.

Returns a copy of the dataframe.

- null_counts(df)

Returns a dataframe containing the number of null values in each column of a given dataframe.

### Additional Setup

In [3]:
# to allow for all variables to be displayed in jupyter
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Workflow
Read in the data and take an initial look.

In [12]:
input_loc = '../data/raw/'
trans_df = pd.read_csv(input_loc+'train_transaction.csv')
id_df = pd.read_csv(input_loc+'train_identity.csv')

C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
temp = [x for x in list(trans_df.columns) if x.find('323') >-1]
temp

['V323', 'v323']

In [5]:
len(set(trans_df.columns)) == len(trans_df.columns)

True

In [6]:
len(set(id_df.columns)) == len(id_df.columns)

True

In [7]:
# using the provided functions
trans_df = standardize_col_names(trans_df)
id_df = standardize_col_names(id_df)

In [8]:
len(set(trans_df.columns)) == len(trans_df.columns)

False

In [9]:
len(set(id_df.columns)) == len(id_df.columns)

True

In [11]:
temp = [x for x in list(trans_df.columns) if list(trans_df.columns).count(x) >1]
temp

['v5',
 'v5',
 'v12',
 'v12',
 'v42',
 'v42',
 'v139',
 'v139',
 'v157',
 'v157',
 'v298',
 'v298',
 'v301',
 'v301',
 'v323',
 'v323']

In [6]:
# reshaping id data
id_df = ngv_reshape_id_data(id_df)

In [7]:
# setting transaction data index
trans_df.set_index('transactionid', drop=True, inplace=True)
# dropping column with all NaNs
trans_df.drop(['v340'], axis=1, inplace=True)

In [8]:
tf_id_bools = ['id_35', 'id_36', 'id_37', 'id_38']
tf_trans_bools = ['m1', 'm2', 'm3', 'm5', 'm6', 'm7', 'm8', 'm9']
tf_map = {'T': True, 'F': False}
found_bools = ['id_12', 'id_16', 'id_27', 'id_28', 'id_29']
found_map = {'Found': True, 'NotFound': False, 'New': False}
device_bools =['DeviceType']
device_map = {'desktop': True, 'mobile': False}

In [9]:
# re-encoding columns
id_df = ngv_reencode_match_status(id_df)

trans_df = ngv_reencode_mapping(trans_df, tf_trans_bools, tf_map)
id_df = ngv_reencode_mapping(id_df, tf_id_bools, tf_map)
id_df = ngv_reencode_mapping(id_df, found_bools, found_map)
id_df = ngv_reencode_mapping(id_df, device_bools, device_map)

In [ ]:
trans_df = trans_df.loc[:, ~trans_df.columns.duplicated()]

In [10]:
# merging the data
tot_df = trans_df.merge(id_df, how='left', left_index=True, right_index=True)

In [4]:
# trans_path = '../data/raw/train_transaction.csv'
# id_path = '../data/raw/train_identity.csv'
# total_df = ngv_initial_data_setup(trans_path, id_path)

C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
# total_df.head()

,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,p_emaildomain,r_emaildomain,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,m1,m2,m3,m4,m5,m6,m7,m8,m9,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v289,v290,v291,v292,v293,v294,v295,v296,v297,v298,v299,v300,v301,v302,v303,v304,v305,v306,v307,v308,v309,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,transactionamt_long_dec,DeviceInfo,DeviceType,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,id_30_name,id_31_bin_many,id_31_bin_few,id_33_prod,id_33_ratio,DeviceInfo_trunc,DeviceInfo_alpha_trunc
transactionid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,4,13926,NaN,150.0,0.0,142.0,2.0,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [7]:
output_loc = '../data/interim/'
file_name = 'ngv_analytic.csv'
file_name_mod = 'ngv_modeling.csv'
# tot_df.to_csv(output_loc+file_name)
# total_df.to_csv(output_loc+file_name_mod)

In [14]:
total_df.head()

,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,p_emaildomain,r_emaildomain,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,m1,m2,m3,m4,m5,m6,m7,m8,m9,v1,v2,v3,v4,v5,v5,v6,v7,v8,v9,v10,v11,v12,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v289,v290,v291,v292,v293,v294,v295,v296,v297,v298,v298,v299,v300,v301,v301,v302,v303,v304,v305,v306,v307,v308,v309,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,transactionamt_long_dec,DeviceInfo,DeviceType,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,id_30_name,id_31_bin_many,id_31_bin_few,id_33_prod,id_33_ratio,DeviceInfo_trunc,DeviceInfo_alpha_trunc
transactionid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,4,13926,NaN,150.0,0.0,142.0,2.0,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [15]:
# confirmed duplicate column names are in fact duplicates for all their data

# dup_cols = tot_df.columns.value_counts() > 1
# dup_cols = dup_cols.loc[dup_cols == True]
# dup_cols = list(dup_cols.index)

In [16]:
# dup_df = tot_df.loc[:, dup_cols].T
# dup_df

In [17]:
# dup_df.drop_duplicates()

## Describing the Data
Before diving into the data, let's consider a few practical issues, which may or may not be important, depending on your dataset.

In [18]:
input_loc = '../data/raw/'
trans_df = pd.read_csv(input_loc+'train_transaction.csv')
id_df = pd.read_csv(input_loc+'train_identity.csv')

C:\tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
# using the provided functions
trans_df = standardize_col_names(trans_df)
id_df = standardize_col_names(id_df)

### Data Shape
Take a look at the current shape of the data set or sets. Is it in a convenient form to perform some exploratory analysis on? If not, what will you need to change before analysis can happen?

In [20]:
print(trans_df.shape)
print(id_df.shape)

(590540, 403)
(5769320, 3)


### Data Type
Which data types does each variable contain? Are variables continuous or categorical?

### Data Amount
How much data does the set contain? (ie, how many variables, how many records, how many files?) If the data set is very large, this may effect the tools you can use later (or else how much features selection will need to take place).

In [21]:
trans_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 403 columns):
 #   Column          Dtype  
---  ------          -----  
 0   transactionid   int64  
 1   isfraud         int64  
 2   transactiondt   int64  
 3   transactionamt  float64
 4   productcd       object 
 5   card1           int64  
 6   card2           float64
 7   card3           float64
 8   card4           object 
 9   card5           float64
 10  card6           object 
 11  addr1           float64
 12  addr2           float64
 13  dist1           float64
 14  dist2           float64
 15  p_emaildomain   object 
 16  r_emaildomain   object 
 17  c1              float64
 18  c2              float64
 19  c3              float64
 20  c4              float64
 21  c5              float64
 22  c6              float64
 23  c7              float64
 24  c8              float64
 25  c9              float64
 26  c10             float64
 27  c11             float64
 28  c12          

In [22]:
id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5769320 entries, 0 to 5769319
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   transactionid  int64 
 1   variable       object
 2   value          object
dtypes: int64(1), object(2)
memory usage: 132.0+ MB


In [23]:
trans_df.head()

,transactionid,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,p_emaildomain,r_emaildomain,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,m1,m2,m3,m4,m5,m6,m7,m8,m9,v1,v2,v3,v4,v5,v5,v6,v7,v8,v9,v10,v11,v12,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v289,v290,v291,v292,v293,v294,v295,v296,v297,v298,v298,v299,v300,v301,v301,v302,v303,v304,v305,v306,v307,v308,v309,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,v340
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0

In [24]:
id_df.head()

,transactionid,variable,value
0,2987004,id_01,0
1,2987008,id_01,-5
2,2987010,id_01,-5
3,2987011,id_01,-5
4,2987016,id_01,0


In [25]:
# No duplicate records
# There are some records that are duplicates (or at least have all the same data) if the TransactionID is excluded
print('Trans Duplicate Count = ', trans_df.shape[0] - trans_df.drop_duplicates().shape[0])
print('ID Duplicate Count = ', id_df.shape[0] - id_df.drop_duplicates().shape[0])

# No duplicate transaction IDs
print('Total Trans Transaction_IDs ', trans_df['transactionid'].shape[0])
print('Unique Trans Transaction_IDs ', len(trans_df['transactionid'].unique()))
print('Total ID Transaction_IDs per Variable ', int(id_df['transactionid'].shape[0]/len(id_df['variable'].unique())))
print('Unique ID Transaction_IDs ', len(id_df['transactionid'].unique()))

Trans Duplicate Count =  0
ID Duplicate Count =  0
Total Trans Transaction_IDs  590540
Unique Trans Transaction_IDs  590540
Total ID Transaction_IDs per Variable  144233
Unique ID Transaction_IDs  144233


## Cleaning

Before you can dive into exploratory analysis, it's likely that you will have to do some reshaping of the data. Depending on what your data set looks like, there may be a lot or very little work to do here.

### Combine and/or Reshape
Be careful not to lose any data while reshaping. Make sure you have the same number of variables and observations in the raw set as you do after reshaping. Keep in mind that, for analysis and machine learning, you will want to have one or more dataframes with each column representing a measured variable and each row representing an observation.

## Handling Identify Table Reshaping and DataTypes

In [26]:
# Reshaping the identity table to make each variable its own column
# checking initial sizes
print(len(id_df['variable'].unique()))
print(len(id_df['transactionid'].unique()))

40
144233


In [27]:
# reshaping/pivoting
id_df = id_df.pivot(index='transactionid', columns='variable', values='value')

In [28]:
id_df = id_df.apply(pd.to_numeric, errors='ignore', downcast='integer')

In [29]:
# inspecting reshaped df
print(id_df.shape)
id_df.head()

(144233, 40)


variable,DeviceInfo,DeviceType,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
transactionid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987004,SAMSUNG SM-G892A Build/NRD90M,mobile,0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T
2987008,iOS Device,mobile,-5,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T
2987010,Windows,desktop,-5,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T
2987011,NaN,desktop,-5,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T
2987016,MacOS,desktop,0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T


### Indexing
Which variable or variables should you index on? Depending on your dataset, this may or may not have a simple answer. Regardless, choose one or more index values for your data set.

In [30]:
# can index on transaction id

In [31]:
trans_df.set_index('transactionid', drop=True, inplace=True)

### Variable Names
Depending on your data set, variables may have names with strange symbols, which can making loading, saving, and subsetting data difficult. If applicable, you should deal with this now.

In [32]:
# resolved above with provided standardize_col_names() function

### Duplicates
Deal with any duplicate rows/columns (or, contrarily, any empty rows/columns), if applicable.

In [33]:
# see above

### Categorical Encoding
Are there any categorical variables in your data? If so, how would you deal with this so that it can be handled by a machine learning algorithm? You're free to implement the solution now or during your model training process, whichever suits you better.

In [34]:
# first for the identity table

In [35]:
# inspecting potential values of various object columns
id_obj_cols = id_df.select_dtypes('object').columns
for col in id_obj_cols:
    print(col+"'s count of unique values ",len(id_df[col].value_counts()))

DeviceInfo's count of unique values  1786
DeviceType's count of unique values  2
id_12's count of unique values  2
id_15's count of unique values  3
id_16's count of unique values  2
id_23's count of unique values  3
id_27's count of unique values  2
id_28's count of unique values  2
id_29's count of unique values  2
id_30's count of unique values  75
id_31's count of unique values  130
id_33's count of unique values  260
id_34's count of unique values  4
id_35's count of unique values  2
id_36's count of unique values  2
id_37's count of unique values  2
id_38's count of unique values  2


In [36]:
# now looking at those with only 3-4 values
# clear re-encoding opportunity for id_34 to just retain the trailing number
for col in id_obj_cols:
    items = len(id_df[col].value_counts())
    if (items == 3) | (items == 4):
        print(id_df[col].value_counts())

Found      67728
New        61612
Unknown    11645
Name: id_15, dtype: int64
IP_PROXY:TRANSPARENT    3489
IP_PROXY:ANONYMOUS      1071
IP_PROXY:HIDDEN          609
Name: id_23, dtype: int64
match_status:2     60011
match_status:1     17376
match_status:0       415
match_status:-1        3
Name: id_34, dtype: int64


In [37]:
# stripping 'match_status:' to make that variable reflect the number in its strings
id_df['id_34'] = id_df['id_34'].str.strip('match_status:')
id_df['id_34'] = pd.to_numeric(id_df['id_34'], downcast='integer')

In [38]:
# now looking at those with only 2 values to identify booleans
for col in id_obj_cols:
    items = len(id_df[col].value_counts())
    if items == 2:
        print(id_df[col].value_counts())

desktop    85165
mobile     55645
Name: DeviceType, dtype: int64
NotFound    123025
Found        21208
Name: id_12, dtype: int64
Found       66324
NotFound    63016
Name: id_16, dtype: int64
Found       5155
NotFound      14
Name: id_27, dtype: int64
Found    76232
New      64746
Name: id_28, dtype: int64
Found       74926
NotFound    66052
Name: id_29, dtype: int64
T    77814
F    63171
Name: id_35, dtype: int64
F    134066
T      6919
Name: id_36, dtype: int64
T    110452
F     30533
Name: id_37, dtype: int64
F    73922
T    67063
Name: id_38, dtype: int64


In [39]:
# converting T/F booleans
bools = ['id_35', 'id_36', 'id_37', 'id_38']
for col in bools:
    id_df[col] = id_df[col].map({'T': True, 'F': False})

In [40]:
# converting Found booleans
bools = ['id_12', 'id_16', 'id_27', 'id_28', 'id_29']
for col in bools:
    id_df[col] = id_df[col].map({'Found': True, 'NotFound': False, 'New': False})

In [41]:
# converting desktop booleans
bools = ['DeviceType']
for col in bools:
    id_df[col] = id_df[col].map({'desktop': True, 'mobile': False})

In [42]:
# none of the numeric columns appear to be booleans
id_df.describe()

variable,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_13,id_14,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32,id_34
count,144233.000000,140872.000000,66324.000000,66324.000000,136865.000000,136865.000000,5155.000000,5155.000000,74926.000000,74926.000000,140978.000000,127320.000000,80044.000000,139369.000000,45113.000000,139318.000000,139261.000000,5159.000000,5169.000000,4747.000000,5132.000000,5163.000000,77586.000000,77805.000000
mean,-10.170502,174716.584708,0.060189,-0.058938,1.615585,-6.698710,13.285354,-38.600388,0.091023,-0.301124,99.745325,48.053071,-344.507146,189.451377,14.237337,353.128174,403.882666,368.269820,16.002708,12.800927,329.608924,149.070308,26.508597,1.765889
std,14.347949,159651.816856,0.598231,0.701015,5.249856,16.491104,11.384207,26.084899,0.983842,2.789446,1.127602,11.774858,93.695502,30.375360,1.561302,141.095343,152.160327,198.847038,6.897665,2.372447,97.461089,32.101995,3.737502,0.436124
min,-100.000000,1.000000,-13.000000,-28.000000,-72.000000,-100.000000,-46.000000,-100.000000,-36.000000,-100.000000,90.000000,10.000000,-660.000000,100.000000,10.000000,100.000000,100.000000,100.000000,10.000000,11.000000,100.000000,100.000000,0.000000,-1.000000
25%,-10.000000,67992.000000,0.000000,0.000000,0.000000,-6.000000,5.000000,-48.000000,0.000000,0.000000,100.000000,49.000000,-360.000000,166.000000,13.000000,266.000000,256.000000,252.000000,14.000000,11.000000,321.000000,119.000000,24.000000,2.000000
50%,-5.000000,125800.500000,0.000000,0.000000,0.000000,0.000000,14.000000,-34.000000,0.000000,0.000000,100.000000,52.000000,-300.000000,166.000000,15.000000,341.000000,472.000000,252.000000,14.000000,11.000000,321.000000,149.000000,24.000000,2.000000
75%,-5.000000,228749.000000,0.000000,0.000000,1.000000,0.000000,22.000000,-23.000000,0.000000,0.000000,100.000000,52.000000,-300.000000,225.000000,15.000000,427.000000,533.000000,486.500000,14.000000,15.000000,371.000000,169.000000,32.000000,2.000000
max,0.000000,999595.000000,10.000000,0.000000,52.000000,0.000000,61.000000,0.000000,25.000000,0.000000,100.000000,64.000000,720.000000,229.000000,29.000000,671.000000,661.000000,854.000000,44.000000,26.000000,548.000000,216.000000,32.000000,2.000000


In [43]:
# DeviceInfo, id_30, and id_31 look like they could potentially benefit from binning
# id_33 look like it could be re-encoded to a number, appears to be a resolution or something
id_df.head()

variable,DeviceInfo,DeviceType,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
transactionid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987004,SAMSUNG SM-G892A Build/NRD90M,False,0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,False,NaN,-480.0,New,False,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,Android 7.0,samsung browser 6.2,32.0,2220x1080,2.0,True,False,True,True
2987008,iOS Device,False,-5,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,False,49.0,-300.0,New,False,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,1.0,True,False,False,True
2987010,Windows,True,-5,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,False,52.0,NaN,Found,True,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,chrome 62.0,NaN,NaN,NaN,False,False,True,True
2987011,NaN,True,-5,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,False,52.0,NaN,New,False,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN,chrome 62.0,NaN,NaN,NaN,False,False,True,True
2987016,MacOS,True,0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,False,NaN,-300.0,Found,True,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,2.0,True,False,True,True


In [44]:
id_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144233 entries, 2987004 to 3577534
Data columns (total 40 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   DeviceInfo  118666 non-null  object 
 1   DeviceType  140810 non-null  object 
 2   id_01       144233 non-null  int8   
 3   id_02       140872 non-null  float64
 4   id_03       66324 non-null   float64
 5   id_04       66324 non-null   float64
 6   id_05       136865 non-null  float64
 7   id_06       136865 non-null  float64
 8   id_07       5155 non-null    float64
 9   id_08       5155 non-null    float64
 10  id_09       74926 non-null   float64
 11  id_10       74926 non-null   float64
 12  id_11       140978 non-null  float64
 13  id_12       144233 non-null  bool   
 14  id_13       127320 non-null  float64
 15  id_14       80044 non-null   float64
 16  id_15       140985 non-null  object 
 17  id_16       129340 non-null  object 
 18  id_17       139369 non-null  float64


In [45]:
# Now for the transaction data

In [46]:
trans_df.head()

,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,p_emaildomain,r_emaildomain,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,m1,m2,m3,m4,m5,m6,m7,m8,m9,v1,v2,v3,v4,v5,v5,v6,v7,v8,v9,v10,v11,v12,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v289,v290,v291,v292,v293,v294,v295,v296,v297,v298,v298,v299,v300,v301,v301,v302,v303,v304,v305,v306,v307,v308,v309,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,v340
transactionid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,cred

In [47]:
# inspecting potential values of various object columns
# need to look into binning or re-encoding emaildomain if kept
trans_obj_cols = trans_df.select_dtypes('object').columns
for col in trans_obj_cols:
    print(col+"'s count of unique values ", len(trans_df[col].value_counts()))

productcd's count of unique values  5
card4's count of unique values  4
card6's count of unique values  4
p_emaildomain's count of unique values  59
r_emaildomain's count of unique values  60
m1's count of unique values  2
m2's count of unique values  2
m3's count of unique values  2
m4's count of unique values  3
m5's count of unique values  2
m6's count of unique values  2
m7's count of unique values  2
m8's count of unique values  2
m9's count of unique values  2


In [48]:
# now looking at those with only 3-5 values
# no clear re-encoding opportunities without further inspection
for col in trans_obj_cols:
    items = len(trans_df[col].value_counts())
    if (items > 2) & (items < 6):
        print(trans_df[col].value_counts())

W    439670
C     68519
R     37699
H     33024
S     11628
Name: productcd, dtype: int64
visa                384767
mastercard          189217
american express      8328
discover              6651
Name: card4, dtype: int64
debit              439938
credit             148986
debit or credit        30
charge card            15
Name: card6, dtype: int64
M0    196405
M2     59865
M1     52826
Name: m4, dtype: int64


In [49]:
# now looking at those with only 2 values
trans_boo1_cols = []
for col in trans_obj_cols:
    items = len(trans_df[col].value_counts())
    if items == 2:
        print(trans_df[col].value_counts())
        trans_boo1_cols.append(col)

T    319415
F        25
Name: m1, dtype: int64
T    285468
F     33972
Name: m2, dtype: int64
T    251731
F     67709
Name: m3, dtype: int64
F    132491
T    107567
Name: m5, dtype: int64
F    227856
T    193324
Name: m6, dtype: int64
F    211374
T     32901
Name: m7, dtype: int64
F    155251
T     89037
Name: m8, dtype: int64
T    205656
F     38632
Name: m9, dtype: int64


In [50]:
# converting booleans
for col in trans_boo1_cols:
    trans_df[col] = trans_df[col].map({'T': True, 'F': False})

### Missing Data
If missing data is present (ie, NaN values), what is the best way to deal with it? Should values be imputed (and if so, how)? Or should they simply be filled? How will each option will affect the ultimate outcome of your model? Design and implement a solution.

In [51]:
# Not going to impute values because the feature description is obscured and has been engineered from other features,
# so a NaN may represent that the underlying feature can't exist, plus there's just so many.

# The V columns are stated to be ranking, counting, etc. features,
# so replacing their Nan's with a really low number (i.e. -9999) should allow 
# for more effective decision thresholds (I'm planning to use a decision tree model) than leaving as NaNs.
# Other types may get a similar treatment after further review - will make those replacements after EDA

# Many of the d-variables and the dist2 variable have super high null counts,
# I'll likely define a threshold of Nan's a column can have before it is dropped.

# lots of variables with the same number of NaNs.
# Since there's a pattern, and so many variables in general, 
# I'll plan on retaining only a few variables with a given # of NaNs,
# especially if the null_fraction is very high.

In [52]:
null_counts(trans_df)

,null_count,null_fraction
v340,590540,1.000000
dist2,552913,0.936284
d7,551623,0.934099
d13,528588,0.895093
d14,528353,0.894695
d12,525823,0.890410
d6,517353,0.876068
d8,515614,0.873123
d9,515614,0.873123
v146,508595,0.861237


In [53]:
# many of these have high null_fractions which will only increase after 

In [54]:
null_counts(id_df)

,null_count,null_fraction
variable,,
id_24,139486,0.967088
id_25,139101,0.964419
id_07,139078,0.964259
id_08,139078,0.964259
id_21,139074,0.964231
id_26,139070,0.964204
id_23,139064,0.964162
id_27,139064,0.964162
id_22,139064,0.964162


# Merging the data sets

In [55]:
id_ids = set(id_df.index)
trans_ids = set(trans_df.index)
print(len(id_ids))
print(len(trans_ids))

144233
590540


In [56]:
# all of the Identity tables transactions exist in the transactions table, but not the other way around
ids_union = id_ids.union(trans_ids)
ids_inter = id_ids.intersection(trans_ids)
print('union len ', len(ids_union))
print('intersection len ', len(ids_inter))

union len  590540
intersection len  144233


In [57]:
tot_df = trans_df.merge(id_df, how='left', left_index=True, right_index=True)
tot_df.head()

,isfraud,transactiondt,transactionamt,productcd,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,p_emaildomain,r_emaildomain,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,m1,m2,m3,m4,m5,m6,m7,m8,m9,v1,v2,v3,v4,v5,v5,v6,v7,v8,v9,v10,v11,v12,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v289,v290,v291,v292,v293,v294,v295,v296,v297,v298,v298,v299,v300,v301,v301,v302,v303,v304,v305,v306,v307,v308,v309,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,v340,DeviceInfo,DeviceType,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
transactionid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,True,True,True,M2,False,True,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0

In [58]:
print('total duplicate count = ', tot_df.shape[0] - tot_df.drop_duplicates().shape[0])

total duplicate count =  3


### Uniformity
Are all variables measured using compatible units? Or are there monetary values in different forms of currency? Or something else entirely? If any of these issues are applicable to your data, design and implement a solution.

In [59]:
# Nothing jumps out, and since I plan on using a decision tree model, scaling won't be an issue.

In [60]:
tot_df.describe()

,isfraud,transactiondt,transactionamt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,v1,v2,v3,v4,v5,v5,v6,v7,v8,v9,v10,v11,v12,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,v36,v37,v38,v39,v40,v41,v42,v42,v43,v44,v45,v46,v47,v48,v49,v50,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60,v61,v62,v63,v64,v65,v66,v67,v68,v69,v70,v71,v72,v73,v74,v75,v76,v77,v78,v79,v80,v81,v82,v83,v84,v85,v86,v87,v88,v89,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,v139,v139,v140,v141,v142,v143,v144,v145,v146,v147,v148,v149,v150,v151,v152,v153,v154,v155,v156,v157,v157,v158,v159,v160,v161,v162,v163,v164,v165,v166,v167,v168,v169,v170,v171,v172,v173,v174,v175,v176,v177,v178,v179,v180,v181,v182,v183,v184,v185,v186,v187,v188,v189,v190,v191,v192,v193,v194,v195,v196,v197,v198,v199,v200,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255,v256,v257,v258,v259,v260,v261,v262,v263,v264,v265,v266,v267,v268,v269,v270,v271,v272,v273,v274,v275,v276,v277,v278,v279,v280,v281,v282,v283,v284,v285,v286,v287,v288,v289,v290,v291,v292,v293,v294,v295,v296,v297,v298,v298,v299,v300,v301,v301,v302,v303,v304,v305,v306,v307,v308,v309,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,v340,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_13,id_14,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32,id_34
count,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,238269.000000,37627.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,590540.000000,589271.000000,309743.000000,327662.000000,421618.000000,280699.000000,73187.000000,38917.000000,74926.000000,74926.000000,514518.000000,311253.000000,64717.000000,61952.000000,62187.000000,501427.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,311253.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,421571.000000,421571.000000,421571.000000,421571.00000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,590226.000000,590226.00000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,59022

### Additional Cleaning
Depending on your data set, there may be additional cleaning you would like to do at this stage. If so, do that here.

In [61]:
# not at this stage

####  Save the Analytic Set
After this step, you should have a single dataframe with any inconsistencies, such as non-uniform column names and missing data, fixed. Depending on the data, the dataframe could be multiindexed as well. This is a good time to save the set. We'll load it from here during the next notebook.

Be sure to give your analytic set a unique name, as other people will be using the same repository to store their data on git. To adhere to the git repo naming conventions, prepend your initials to the filename.

In [62]:
# output_loc = '../data/interim/'
# file_name = 'ngv_cleaned_data.csv'
# tot_df.to_csv(output_loc+file_name)

## Verifying the Data Quality
While we're cleaning the data, it's important to give some thought to the quality.

### Data Source and Reliability
What is the origin of the data? How was it obtained? For quality assurance purposes, it's relevant to know how reliable the data is. This will help you deal with any potential data-entry errors or determine how trustworthy your eventual model will be.

In [63]:
# As a major company utilizing Kaggle, I'd imagine the data is relatively high quality, 
# but appears to have lots of NaNs, likely due to inherent unavailability of certain data

### Error Handling
Are there any data-entry errors in the data? How can you tell? How would you characterize the overall quality of the dataset?
If appropriate, implement a solution for dealing with any errors you find.

In [64]:
# didn't notice any at this stage

## Outcome 

By the end of this notebook you should have an analytic data set and be ready to dive into some analysis.